In [80]:
import pynq
import collections

from pynq import Overlay, UnsupportedConfiguration
from pynq import Bitstream
from pprint import pprint
import numpy as np
from tqdm import tqdm
import time
import re

def empty_1_switch():
    time_list_1 = []
    CONTROL_REGISTER = 0x0

    N = 1
    DIM = 1
    buff_x0    = pynq.allocate(shape=(N,), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)
    buff_x0[:] = [1]

    ad_ip1 = ol.pr1.pr1_empty_ad
    combo_ip1 = ol.pr_combo_1.pr_combo_1_empty_combo
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel  
    dma_y0 = ol.axi_dma_in0.recvchannel

    axiswitch_ip1.write(0x5c, 0x00) # m7-s0

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    for i in range(1):
        ######################################  Empty Test ######################################
        ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
        combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0

        start_ = time.time()

        dma_x0.transfer(buff_x0)
        dma_y0.transfer(buff_y0)

        dma_x0.wait()
        dma_y0.wait()

        end_ = time.time()
        ad_ip1.write(CONTROL_REGISTER,0x00) # stop
        combo_ip1.write(CONTROL_REGISTER,0x00) # stop

        time_cost= end_ - start_
        print('empty timecost(ms) = ',time_cost*1000)
        del buff_x0, buff_y0
        buff_x0    = pynq.allocate(shape=(N,), dtype=np.float32)
        buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)
        buff_x0[:] = [1]

        time_list_1.append(time_cost*1000)
        print('i = ',i)
    print('averaging timecost(ms) = ',np.mean(np.array(time_list_1)))

def empty_2_switch():
    time_list = []
    CONTROL_REGISTER = 0x0

    N = 1
    DIM = 1
    buff_x0    = pynq.allocate(shape=(N,), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)
    buff_x0[:] = [1]

    ad_ip1 = ol.pr1.pr1_empty_ad
    combo_ip1 = ol.pr_combo_1.pr_combo_1_empty_combo
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel  
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0-s0
    axiswitch_ip1.write(0x5c, 0x07) # m7-s7

    axiswitch_ip2.write(0x40, 0x00) # m0-s0
    axiswitch_ip2.write(0x70, 0x07) # m12-s7

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ######################################  Empty Test ######################################
    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0

    start_ = time.time()

    dma_x0.transfer(buff_x0)
    dma_y0.transfer(buff_y0)

    dma_x0.wait()
    dma_y0.wait()

    end_ = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER,0x00) # stop

    time_cost= end_ - start_
    print('empty timecost(ms) = ',time_cost*1000)
    del buff_x0, buff_y0


In [81]:
ol = Overlay('./system.bit')
ol.download()
ol.is_loaded()

decouper_1 = ol.dfx_decoupler_1
decouper_1.write(0x0,0)
decouper_1.write(0x0,1)
ol.pr_download('pr1','pr1_empty_ad.bit','s_axi_AXILiteS', 'pr1_empty_ad',None)
decouper_1.write(0x0,0)

decouper_combo = ol.dfx_decoupler_combo1
decouper_combo.write(0x0,0)
decouper_combo.write(0x0,1)
ol.pr_download('pr_combo_1','pr_combo_1_empty_combo.bit','s_axi_AXILiteS', 'pr_combo_1_empty_combo',None)
decouper_combo.write(0x0,0)

ad_ip1 = ol.pr1.pr1_empty_ad
combo_ip1 = ol.pr_combo_1.pr_combo_1_empty_combo
axiswitch_ip1 = ol.axis_switch
axiswitch_ip2 = ol.axis_switch1

dma_x0 = ol.axi_dma_in0.sendchannel  
dma_y0 = ol.axi_dma_in0.recvchannel

empty_1_switch()

empty timecost(ms) =  0.3676414489746094
i =  0
averaging timecost(ms) =  0.3676414489746094


In [82]:
ol = Overlay('./system.bit')
ol.download()
ol.is_loaded()

decouper_1 = ol.dfx_decoupler_1
decouper_1.write(0x0,0)
decouper_1.write(0x0,1)
ol.pr_download('pr1','pr1_empty_ad.bit','s_axi_AXILiteS', 'pr1_empty_ad',None)
decouper_1.write(0x0,0)

decouper_combo = ol.dfx_decoupler_combo1
decouper_combo.write(0x0,0)
decouper_combo.write(0x0,1)
ol.pr_download('pr_combo_1','pr_combo_1_empty_combo.bit','s_axi_AXILiteS', 'pr_combo_1_empty_combo',None)
decouper_combo.write(0x0,0)

ad_ip1 = ol.pr1.pr1_empty_ad
combo_ip1 = ol.pr_combo_1.pr_combo_1_empty_combo
axiswitch_ip1 = ol.axis_switch
axiswitch_ip2 = ol.axis_switch1

dma_x0 = ol.axi_dma_in0.sendchannel  
dma_y0 = ol.axi_dma_in0.recvchannel
empty_2_switch()

empty timecost(ms) =  0.3745555877685547
